In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import ipywidgets as widgets
from matplotlib.ticker import EngFormatter
from matplotlib.colors import ListedColormap
import matplotlib as mpl
import re
import numpy as np
from jinja2 import Template
from subprocess import check_call
import concurrent.futures
import multiprocessing as mp
import glob

In [2]:
# Reading Pad Documentation csv file to extract parameters
doc_df = pd.read_csv("pad_documentation/Pad_documentation _for _SKY130_ MPW_Manufacturing.csv")
nfet_01v8_df = doc_df.loc[doc_df["New Style Name"] == "nfet_01v8"]

In [3]:
# curve_type must be one of ['ID_VDS', 'ID_VGS']
curve_type = "ID_VDS"

In [20]:
# Generating sim data
def simulate_device(netlist_path):
    check_call(f"ngspice -b {netlist_path} -o output.log  ", shell= True)

workers_count = 2 * mp.cpu_count()
nmos_netlist = "templates/nfet_01v8_template.cir"
loc_sweep = list()
s_pin_sweep = list()
d_pin_sweep = list()

if curve_type == "ID_VDS":
    sim_cmd = "DC vds 0 1.8 0.05"

elif curve_type == "ID_VGS":
    sim_cmd = "DC vgs 0 1.8 0.05"

with concurrent.futures.ProcessPoolExecutor(max_workers=workers_count) as executor:
    for i, row in nfet_01v8_df.iterrows():
        sim_data = row["Description"]
        subs = [';', 'nshort', 'in DNW', 'contact-gate=\d+.\d+um']
        for element in subs:
            sim_data = re.sub(element, ' ' ,sim_data)

        # get pins loc for each file
        pin1 = int()
        pin2 = int()
        for c in row.index:
            if row[c] == "s":
                pin1 = int(re.sub("Pin ", '', c))
            elif row[c] == "d":
                pin2 = int(re.sub("Pin ", '', c))

        # saving sweep data
        s_pin_sweep.append(int(pin1))
        d_pin_sweep.append(int(pin2))
        if row["Mod #"] == row["Mod #"]:
            loc_sweep.append(int(row["Mod #"]))
        
        # get width, length, multiplier
        width = re.findall("w=\d*.\d*", sim_data)[0]
        width = re.sub("=", "", width)
        length = re.findall("l=\d*.\d*", sim_data)[0]
        length = re.sub("=", "", length)
        multiplier = re.findall("m=\d*", sim_data)[0]
        multiplier = re.sub("=", "", multiplier)

        # naming circuit, csv files
        file_name = f"nfet_01v8_{loc_sweep[-1]}_{pin1}_{pin2}"
        netlist_path = f"{file_name}.cir"
        csv_path = f"{file_name}.csv"

        with open(nmos_netlist) as f:
                    tmpl = Template(f.read())
                    with open(netlist_path, "w") as netlist:
                        netlist.write(
                            tmpl.render(
                                csv_path=csv_path,
                                dimensions=sim_data,
                                sim_cmd=sim_cmd,
                            )
                        )
        
        executor.submit(simulate_device, netlist_path)

In [5]:
# clean unused files
check_call("rm -f nfet_*.cir", shell=True)
check_call("rm -f output.log", shell=True)

0

In [6]:
# Generating raw_data
#check_call(f"python3 raw_gen.py --curve_type={curve_type}", shell=True)
raw_df = pd.read_csv("raw_data.csv")

In [25]:
# widgets settings 
loc_sweep = [*set(loc_sweep)]
s_pin_sweep = [*set(s_pin_sweep)]
d_pin_sweep = [*set(d_pin_sweep)]

loc = widgets.Dropdown(
    options=loc_sweep,
    value=loc_sweep[0],
    description='loc:',
    disabled=False,
)
s_pin = widgets.Dropdown(
    options=s_pin_sweep,
    value=s_pin_sweep[0],
    description='s-pin:',
    disabled=False,
)
d_pin = widgets.Dropdown(
    options=d_pin_sweep,
    value=d_pin_sweep[0],
    description='d-pin:',
    disabled=False,
)

In [26]:
blue_range =  ListedColormap(mpl.cm.Blues(np.linspace(0.3, 0.8, 256)))
red_range = ListedColormap(mpl.cm.Reds(np.linspace(0.3, 0.8, 256)))

sim_files = glob.glob(f"nfet_01v8*.csv")
def get_sim_df(loc, s_pin, d_pin):

    for file in sim_files:
        file_sp = file.split("_")
        d_read = int(file_sp[4].split(".")[0])
        
        if loc == int(file_sp[2]) and s_pin == int(file_sp[3]) and d_pin == d_read:
            csv_file = file
            print("test")
            break
    sim_df = pd.read_csv(csv_file, delimiter=r"\s+") 
    sim_df.drop(
               sim_df.columns[[x for x in range(0, sim_df.shape[1]) if x % 2 == 0 and x != 0]],
               inplace=True,
               axis=1,
            )
    sim_df.columns = [
        "VDS",
        "VGS",
        "VSB",
        "ID",
    ]
    return sim_df

# plotting data:
def plot(loc, s_pin, d_pin):
    tdf_sim= get_sim_df(loc, s_pin, d_pin)
    tdf_raw = raw_df.loc[(raw_df["loc"] == loc) &  (raw_df["s_pin"] == s_pin)
                 & (raw_df["d_pin"] == d_pin)]

    id_df_sim = tdf_sim[["VDS", "VGS", "ID"]].copy()
    id_df_raw = tdf_raw[["VDS", "VGS", "ID"]].copy()
    
    if curve_type == "ID_VDS":
        id_df_sim.set_index("VDS", inplace=True)
        id_df_sim.sort_index(inplace=True)
        id_df_raw.set_index("VDS", inplace=True)
        id_df_raw.sort_index(inplace=True)
        id_df_sim = pd.pivot_table(id_df_sim.reset_index(),
                                   values="ID",
                                   index="VDS",
                                   columns=["VGS"])
        ax = id_df_sim.plot(colormap=blue_range, grid=True, figsize=(12,6))
        id_df_raw = pd.pivot_table(id_df_raw.reset_index(),
                                   values="ID",
                                   index="VDS",
                                   columns=["VGS"])
        id_df_raw.plot(colormap=red_range, ax=ax)
        plt.legend(title= "VGS", loc=0)
        volt_formatter = EngFormatter(unit='V')
        amp_formatter = EngFormatter(unit='A')
        ax.xaxis.set_major_formatter(volt_formatter)
        ax.yaxis.set_major_formatter(amp_formatter)
        plt.title(f"IDS vs VDS - Simulated (Blues) - Measured (Reds)")
    elif curve_type == "ID_VGS":
        id_df_sim.set_index("VGS", inplace=True)
        id_df_sim.sort_index(inplace=True)
        id_df_raw.set_index("VGS", inplace=True)
        id_df_raw.sort_index(inplace=True)
        id_df_sim = pd.pivot_table(id_df_sim.reset_index(),
                                   values="ID",
                                   index="VGS",
                                   columns=["VDS"])
        ax = id_df_sim.plot(colormap=blue_range, grid=True, figsize=(12,6))
        id_df_raw = pd.pivot_table(id_df_raw.reset_index(),
                                   values="ID",
                                   index="VGS",
                                   columns=["VDS"])
        id_df_raw.plot(colormap=red_range, ax=ax)
        plt.legend(title= "VDS", loc=0)
        volt_formatter = EngFormatter(unit='V')
        amp_formatter = EngFormatter(unit='A')
        ax.xaxis.set_major_formatter(volt_formatter)
        ax.yaxis.set_major_formatter(amp_formatter)
        plt.title(f"IDS vs VGS - Simulated (Blues) - Measured (Reds)")
    plt.ylabel("IDS")
    
    
# interactive mode
widgets.interact(plot, loc=loc, s_pin=s_pin, d_pin=d_pin)

interactive(children=(Dropdown(description='loc:', options=(8008, 8392, 5290, 8434, 8436, 8701), value=8008), …

<function __main__.plot(loc, s_pin, d_pin)>